# HTMX Integration

> HTMX patterns and helpers for daisyUI components in FastHTML

In [ ]:
#| default_exp core.htmx

In [ ]:
#| export
from typing import Dict, Any, Optional, List, Union, Literal, Callable
from dataclasses import dataclass, field
from enum import Enum
from fasthtml.common import *
from cjm_fasthtml_daisyui.core.base import DaisyComponent
from cjm_fasthtml_daisyui.core.colors import SemanticColor

## HTMX Integration Patterns

This module provides HTMX-aware patterns for daisyUI components, embracing the FastHTML philosophy of server-side rendering with progressive enhancement.

In [ ]:
#| export
class HTMXTrigger(str, Enum):
    """Common HTMX trigger events"""
    CLICK = "click"
    CHANGE = "change"
    SUBMIT = "submit"
    LOAD = "load"
    REVEALED = "revealed"
    INTERSECT = "intersect"
    EVERY = "every"
    KEYUP = "keyup"
    FOCUS = "focus"
    BLUR = "blur"
    
    def with_modifier(
        self,
        modifier: str  # TODO: Add description
    ) -> str:  # TODO: Add return description
        """Add modifier to trigger (e.g., 'click once')"""
        return f"{self.value} {modifier}"
    
    def delayed(
        self,
        delay: str  # TODO: Add description
    ) -> str:  # TODO: Add return description
        """Add delay to trigger (e.g., 'keyup delay:500ms')"""
        return f"{self.value} delay:{delay}"
    
    def changed(
        self
    ) -> str:  # TODO: Add return description
        """Add changed modifier (e.g., 'keyup changed')"""
        return f"{self.value} changed"

In [ ]:
#| export
class HTMXSwap(str, Enum):
    """HTMX swap strategies"""
    INNER_HTML = "innerHTML"
    OUTER_HTML = "outerHTML"
    BEFORE_BEGIN = "beforebegin"
    AFTER_BEGIN = "afterbegin"
    BEFORE_END = "beforeend"
    AFTER_END = "afterend"
    DELETE = "delete"
    NONE = "none"
    
    def with_modifier(
        self,
        modifier: str  # TODO: Add description
    ) -> str:  # TODO: Add return description
        """Add swap modifier (e.g., 'innerHTML swap:500ms')"""
        return f"{self.value} {modifier}"
    
    def with_transition(
        self,
        duration: str = "500ms"  # TODO: Add description
    ) -> str:  # TODO: Add return description
        """Add swap transition"""
        return f"{self.value} swap:{duration}"

In [ ]:
#| export
@dataclass
class HTMXAttrs:
    """Container for HTMX attributes"""
    hx_get: Optional[str] = None
    hx_post: Optional[str] = None
    hx_put: Optional[str] = None
    hx_patch: Optional[str] = None
    hx_delete: Optional[str] = None
    hx_trigger: Optional[str] = None
    hx_target: Optional[str] = None
    hx_swap: Optional[str] = None
    hx_indicator: Optional[str] = None
    hx_push_url: Optional[Union[bool, str]] = None
    hx_select: Optional[str] = None
    hx_select_oob: Optional[str] = None
    hx_vals: Optional[Union[str, Dict[str, Any]]] = None
    hx_confirm: Optional[str] = None
    hx_disable: Optional[bool] = None
    hx_disabled_elt: Optional[str] = None
    hx_include: Optional[str] = None
    hx_ext: Optional[str] = None
    
    def to_dict(
        self
    ) -> Dict[str, Any]:  # TODO: Add return description
        """Convert to dictionary of attributes"""
        attrs = {}
        for key, value in self.__dict__.items():
            if value is not None:
                # Convert Python snake_case to HTMX hyphenated attributes
                attr_name = key.replace('_', '-')
                
                # Handle boolean attributes
                if isinstance(value, bool):
                    attrs[attr_name] = str(value).lower()
                # Handle dictionary values (for hx-vals)
                elif isinstance(value, dict):
                    import json
                    attrs[attr_name] = json.dumps(value)
                else:
                    attrs[attr_name] = value
        
        return attrs

## HTMX-Aware Base Component

Extending DaisyComponent with HTMX support:

In [ ]:
#| export
class HTMXComponent(DaisyComponent):
    """
    Base class for HTMX-aware daisyUI components
    
    Extends DaisyComponent with HTMX attributes and helper methods
    for building interactive components following FastHTML patterns.
    """
    
    def __init__(self, *args, htmx: Optional[HTMXAttrs] = None, **kwargs):
        "TODO: Add function description"
        super().__init__(*args, **kwargs)
        self.htmx = htmx or HTMXAttrs()
    
    def with_htmx(
        self,
        get: Optional[str] = None,  # TODO: Add description
        post: Optional[str] = None,  # TODO: Add description
        put: Optional[str] = None,  # TODO: Add description
        patch: Optional[str] = None,  # TODO: Add description
        delete: Optional[str] = None,  # TODO: Add description
        trigger: Optional[Union[HTMXTrigger, str]] = None,
        target: Optional[str] = None,  # TODO: Add description
        swap: Optional[Union[HTMXSwap, str]] = None,
        **kwargs
    ) -> 'HTMXComponent':  # TODO: Add return description
        """
        Configure HTMX attributes fluently
        
        Args:
            get/post/put/patch/delete: URL endpoints
            trigger: Event that triggers the request
            target: CSS selector for target element
            swap: How to swap the response
            **kwargs: Additional HTMX attributes
            
        Returns:
            Self for method chaining
        """
        if get:
            self.htmx.hx_get = get
        if post:
            self.htmx.hx_post = post
        if put:
            self.htmx.hx_put = put
        if patch:
            self.htmx.hx_patch = patch
        if delete:
            self.htmx.hx_delete = delete
        
        if trigger:
            self.htmx.hx_trigger = trigger.value if isinstance(trigger, HTMXTrigger) else trigger
        if target:
            self.htmx.hx_target = target
        if swap:
            self.htmx.hx_swap = swap.value if isinstance(swap, HTMXSwap) else swap
        
        # Handle additional kwargs
        for key, value in kwargs.items():
            if hasattr(self.htmx, f"hx_{key}"):
                setattr(self.htmx, f"hx_{key}", value)
        
        return self
    
    def with_loading(
        self,
        indicator_id: str,  # TODO: Add description
        disable_during: Optional[str] = None  # TODO: Add description
    ) -> 'HTMXComponent':  # TODO: Add return description
        """
        Configure loading indicators
        
        Args:
            indicator_id: ID of the loading indicator element
            disable_during: CSS selector of elements to disable during request
            
        Returns:
            Self for method chaining
        """
        self.htmx.hx_indicator = f"#{indicator_id}"
        if disable_during:
            self.htmx.hx_disabled_elt = disable_during
        return self
    
    def with_confirmation(
        self,
        message: str  # TODO: Add description
    ) -> 'HTMXComponent':  # TODO: Add return description
        """
        Add confirmation dialog
        
        Args:
            message: Confirmation message to show
            
        Returns:
            Self for method chaining
        """
        self.htmx.hx_confirm = message
        return self
    
    def render_attrs(
        self
    ) -> Dict[str, Any]:  # TODO: Add return description
        """Build all HTML attributes including HTMX"""
        attrs = super().render_attrs()
        
        # Add HTMX attributes
        htmx_attrs = self.htmx.to_dict()
        attrs.update(htmx_attrs)
        
        return attrs

## Helper Functions

Utility functions for HTMX integration:

In [ ]:
#| export
def htmx_attrs(
    **kwargs
) -> Dict[str, Any]:  # Dictionary with proper HTMX attribute names
    "Convert keyword arguments to HTMX attributes Converts Python-style names to HTMX attribute names: - get -> hx-get - trigger -> hx-trigger - etc."
    attrs = {}
    for key, value in kwargs.items():
        # Convert to hx- prefix
        if not key.startswith('hx_'):
            key = f'hx_{key}'
        
        # Convert underscores to hyphens
        attr_name = key.replace('_', '-')
        
        # Handle special conversions
        if isinstance(value, HTMXTrigger):
            value = value.value
        elif isinstance(value, HTMXSwap):
            value = value.value
        elif isinstance(value, bool):
            value = str(value).lower()
        elif isinstance(value, dict):
            import json
            value = json.dumps(value)
        
        attrs[attr_name] = value
    
    return attrs

In [ ]:
#| export
def loading_indicator(
    indicator_id: str,  # ID for the indicator
    text: str = "Loading...",  # Loading text
    size: str = "md"  # Size of spinner (xs, sm, md, lg, xl)
) -> FT:  # Loading indicator element
    "Create a loading indicator element"
    return Div(
        Span(cls=f"loading loading-spinner loading-{size}"),
        " ",
        text,
        id=indicator_id,
        cls="htmx-indicator"
    )

In [ ]:
#| export
def oob_alert(
    message: str,  # Alert message
    alert_type: str = "info",  # Type (info, success, warning, error)
    target_id: str = "alerts",  # ID of container to append to
    auto_dismiss: Optional[int] = 5000  # Auto dismiss after milliseconds (None to disable)
) -> FT:  # Alert element with OOB swap
    "Create out-of-band alert message"
    alert_id = f"alert-{id(message)}"
    
    alert = Div(
        message,
        id=alert_id,
        cls=f"alert alert-{alert_type}",
        hx_swap_oob=f"beforeend:#{target_id}"
    )
    
    if auto_dismiss:
        alert.attrs[f"hx-trigger"] = f"load delay:{auto_dismiss}ms"
        alert.attrs["hx-swap"] = "outerHTML"
        alert.attrs["hx-target"] = f"#{alert_id}"
        alert.attrs["hx-get"] = "/empty"  # Endpoint that returns empty response
    
    return alert

## Usage Examples

Examples of using HTMX patterns with daisyUI components:

### Basic HTMX Component

In [ ]:
# Example: Create an HTMX-aware button
class HTMXButton(HTMXComponent):
    def component_class(self) -> str:
        return "btn"

# Create button with HTMX behavior
button = HTMXButton()
button.with_htmx(
    get="/api/data",
    trigger=HTMXTrigger.CLICK,
    target="#results",
    swap=HTMXSwap.INNER_HTML
).with_loading("loading-spinner")

# Get the attributes
attrs = button.render_attrs()
print("Button attributes:")
for key, value in attrs.items():
    print(f"  {key}: {value}")

Button attributes:
  class: btn
  hx-get: /api/data
  hx-trigger: click
  hx-target: #results
  hx-swap: innerHTML
  hx-indicator: #loading-spinner


## Best Practices

1. **Server-Side First**: Embrace FastHTML's server-side rendering philosophy
2. **Progressive Enhancement**: Components work without JavaScript, HTMX enhances them
3. **Semantic HTML**: Use proper HTML elements (dialog, details, form)
4. **Loading States**: Always provide visual feedback during requests
5. **Error Handling**: Use OOB swaps for error messages
6. **Accessibility**: Ensure HTMX enhancements maintain accessibility

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()